<a href="https://colab.research.google.com/github/DavoodSZ1993/Dive_into_Deep_Learning/blob/main/6_builder_guide.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 6.1 Layers and Modules

In [ ]:
import torch
from torch import nn
from torch.nn import functional as F

In [ ]:
net = nn.Sequential(nn.LazyLinear(256),   # input x 256
                    nn.ReLU(),            
                    nn.LazyLinear(10))    # 256 x 10

X = torch.rand(2, 20) # two samples (rows) & 20 features (columns)
X.shape, net(X).shape

(torch.Size([2, 20]), torch.Size([2, 10]))

### 6.1.1 A Custom Module

In [ ]:
class MLP(nn.Module):
  def __init__(self):
    super().__init__()               # Performs necessary initialization from the parent class.
    self.hidden = nn.LazyLinear(256)
    self.out = nn.LazyLinear(10)

  def forward(self, X):              # Does the forward propagation for the given network.
    return self.out(F.relu(self.hidden(X)))

In [ ]:
net = MLP()
net(X).shape

/usr/local/lib/python3.8/dist-packages/torch/nn/modules/lazy.py:180: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


torch.Size([2, 10])

### 6.1.2 The Sequential Module

In [ ]:
class MySequential(nn.Module):
  def __init__(self, *args):
    super().__init__()
    for idx, module in enumerate(args):
      self.add_module(str(idx), module)

  def forward(self, X):
    for module in self.children():
      X = module(X)
    return X

In [ ]:
net_layers = (nn.LazyLinear(256),
              nn.ReLU(),
              nn.LazyLinear(10))
net = MySequential(*net_layers)

net(X).shape

torch.Size([2, 10])

### 6.1.3 Executing Code in the Forward Propagation Method

In [ ]:
class FixedHiddenMLP(nn.Module):
  def __init__(self):
    super().__init__()

    self.rand_weight = torch.rand((20, 20))
    self.linear = nn.LazyLinear(20)

  def forward(self, X):
   
    X = self.linear(X)
    X = F.relu(X @ self.rand_weight + 1)
    X = self.linear(X)

    # Control flow
    while X.abs().sum() > 1:
      X /=2

    return X.sum()

In [ ]:
net = FixedHiddenMLP()
net(X)

/usr/local/lib/python3.8/dist-packages/torch/nn/modules/lazy.py:180: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


tensor(0.0227, grad_fn=<SumBackward0>)

In [ ]:
class NestMLP(nn.Module):
  def __init__(self):
    super().__init__()

    self.net = nn.Sequential(nn.LazyLinear(64),
                             nn.ReLU(),
                             nn.LazyLinear(32),
                             nn.ReLU())
    self.linear = nn.LazyLinear(16)

  def forward(self, X):
    return self.linear(self.net(X))

In [ ]:
chimera = nn.Sequential(NestMLP(),
                        nn.LazyLinear(20),
                        FixedHiddenMLP())

chimera(X)

/usr/local/lib/python3.8/dist-packages/torch/nn/modules/lazy.py:180: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


tensor(0.0920, grad_fn=<SumBackward0>)

## 6.2 Parameter Management

In [1]:
import torch
from torch import nn

X = torch.rand(size=(2, 4))  # two inputs, four features

In [2]:
net = nn.Sequential(nn.LazyLinear(8),
                    nn.ReLU(),
                    nn.LazyLinear(1))

net(X).shape

/usr/local/lib/python3.8/dist-packages/torch/nn/modules/lazy.py:180: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


torch.Size([2, 1])

### 6.2.1 Parameter Access

In [3]:
net[2].state_dict()

OrderedDict([('weight',
              tensor([[ 0.1928, -0.2604,  0.2596,  0.0229,  0.0822, -0.0394, -0.0623, -0.0491]])),
             ('bias', tensor([0.0160]))])

#### Targeted Parameters

In [4]:
type(net[2].bias), net[2].bias.data

(torch.nn.parameter.Parameter, tensor([0.0160]))

In [5]:
net[2].weight.grad == None

True

#### All Parameters at Once

In [6]:
[(name, param.shape) for name, param in net.named_parameters()]

[('0.weight', torch.Size([8, 4])),
 ('0.bias', torch.Size([8])),
 ('2.weight', torch.Size([1, 8])),
 ('2.bias', torch.Size([1]))]

### 6.2.2 Tied Parameters


In [7]:
shared = nn.LazyLinear(8)
net = nn.Sequential(nn.LazyLinear(8), nn.ReLU(),
                    shared, nn.ReLU(),
                    shared, nn.ReLU(),
                    nn.LazyLinear(1))

net(X)

/usr/local/lib/python3.8/dist-packages/torch/nn/modules/lazy.py:180: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


tensor([[-0.3695],
        [-0.3639]], grad_fn=<AddmmBackward0>)

In [10]:
print(net[2].weight.data[0] == net[4].weight.data[0])

net[2].weight.data[0, 0] = 100                        # Changes the weight in both layers.
print(net[2].weight.data[0] == net[4].weight.data[0])

tensor([True, True, True, True, True, True, True, True])
tensor([True, True, True, True, True, True, True, True])
